In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.listdir('.')

['.ipynb_checkpoints',
 'migrated',
 'Base models-entire data.ipynb',
 'Basic models.ipynb',
 'EDA_Challenge.ipynb',
 'EDA.ipynb',
 'EDA_trainData.ipynb',
 'Ensemble models.ipynb',
 'xgboost with null.ipynb',
 'Untitled.ipynb',
 'Base models-clean.ipynb',
 'nohup.out',
 'jupyter_notebook_config.py']

In [58]:
import time
t1=time.time()
train = pd.read_table('/aichallenge/training_set.tsv')
test = pd.read_table('/aichallenge/scoring_set.tsv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,133,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


512.0059087276459


In [59]:
train.columns

Index(['training_data.innovation_challenge_key', 'training_data.renewed_yorn',
       'training_data.instance_id', 'training_data.contract_line_id',
       'training_data.minor_line_yorn', 'training_data.major_line_instance_id',
       'training_data.installation_date',
       'training_data.product_sales_order_type',
       'training_data.product_purchase_order_type',
       'training_data.instance_status',
       ...
       'training_data.service_distributor_base_theater_name',
       'training_data.service_distributor_base_distributor_normalized_name',
       'training_data.contract_line_net_usd_amount',
       'training_data.product_net_price',
       'training_data.sales_node_renewal_rate',
       'training_data.customer_renewal_rate',
       'training_data.partner_renewal_rate',
       'training_data.product_renewal_rate',
       'training_data.service_sales_node_installed_base_sales_node_renewal_rate',
       'training_data.service_partner_installed_base_partner_renewal_rate'],


In [62]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['training_data.renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

KeyError: 'training_data.renewed_yorn'

In [63]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('training_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.0012826919555664062


In [64]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop('renewed_yorn',axis=1)
test_y=test['renewed_yorn']
test = test.drop('renewed_yorn',axis=1)
t2=time.time()
print(t2-t1)

15.580340147018433


In [65]:
#remove columns which have any null values
t1=time.time()
percent = (train.isnull().sum()/train.isnull().count())
train.columns[percent>0.75]
#We would ignore all these 102 columns so we are left with 135 columns
len(train.columns[percent>0.75])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.75]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

58.03571057319641


In [1]:
percent = (train.isnull().sum()/train.isnull().count())
train.columns[percent>0.75]
len(train.columns[percent>0.75])

NameError: name 'train' is not defined

In [66]:
len(unclean_columns)

33

In [67]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in test.columns:
    if(test[col].dtype=='object' and len(test[col].unique())>20):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

20.507105827331543


In [68]:
# 78 messy columns are removed
len(messy_columns)

78

In [69]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

10.147759437561035


In [14]:
train.shape

(6010041, 125)

In [70]:
#Remove the primary key column for data fitting
t1=time.time()
train_ids=train['innovation_challenge_key']
test_ids=test['innovation_challenge_key']
train = train.drop('innovation_challenge_key',axis=1)
test = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

7.753363609313965


In [71]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train.columns:
    if (train[col].dtype !='object'):
        if(len(train[col].unique())<20):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.503035306930542


In [72]:
#handle catcols
t1=time.time()
for col in catcols:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
t2=time.time()
print(t2-t1)

0.6032524108886719


In [73]:
#Replace integer nans with mean 
train[scalcols]=train[scalcols].fillna(train[scalcols].mean())
test[scalcols]=test[scalcols].fillna(test[scalcols].mean())

In [74]:
train[scalcols].isnull().any()

contract_line_net_usd_amount                                 False
product_net_price                                            False
sales_node_renewal_rate                                      False
partner_renewal_rate                                         False
product_renewal_rate                                         False
service_sales_node_installed_base_sales_node_renewal_rate    False
service_partner_installed_base_partner_renewal_rate          False
dtype: bool

In [75]:
test[scalcols].isnull().any()

contract_line_net_usd_amount                                 False
product_net_price                                            False
sales_node_renewal_rate                                      False
partner_renewal_rate                                         False
product_renewal_rate                                         False
service_sales_node_installed_base_sales_node_renewal_rate    False
service_partner_installed_base_partner_renewal_rate          False
dtype: bool

In [76]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[scalcols])
train[scalcols] = scaler.transform(train[scalcols])
scaler = scaler.fit(test[scalcols])
test[scalcols] = scaler.transform(test[scalcols])
t2=time.time()
print(t2-t1)

8.183216571807861


In [77]:
train[scalcols].describe()

,contract_line_net_usd_amount,product_net_price,sales_node_renewal_rate,partner_renewal_rate,product_renewal_rate,service_sales_node_installed_base_sales_node_renewal_rate,service_partner_installed_base_partner_renewal_rate
count,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06
mean,-3.177913e-17,5.735377e-17,-3.990025e-17,-3.949048e-16,-1.751612e-16,-1.080186e-16,-9.717581e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.164629e-01,-2.451377e-01,-9.846433e-02,-2.941421e-01,-1.849635e-01,-8.043840e-02,-6.221420e-02
25%,-1.146668e-01,-2.451377e-01,-3.990931e-17,-3.882947e-02,-1.744921e-16,-1.025815e-02,-8.700442e-03
50%,-1.146668e-01,-2.451377e-01,-3.990931e-17,-3.982183e-16,-1.744921e-16,-1.156315e-16,-9.274100e-18
75%,-1.146668e-01,-1.634183e-01,-3.990931e-17,-3.982183e-16,-1.744921e-16,-1.156315e-16,-9.274100e-18
max,2.899812e+02,7.503942e+01,9.262250e+02,4.837288e+02,2.209099e+02,1.934147e+02,7.311824e+02


In [78]:
train[catcols].describe()

,product_transaction_type,contract_line_reaction_time_code,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level
count,6010041,6010041,6010041,6010041
unique,3,4,6,6
top,10002,-1,6,6
freq,5316273,5985929,5248934,5945627


In [79]:
unclean_cols=train.columns[train.isnull().any()]

In [80]:
#replace all other 43 cols nan with mode values
train[unclean_cols]=train[unclean_cols].fillna(train[unclean_cols].mode())
test[unclean_cols]=test[unclean_cols].fillna(test[unclean_cols].mode())

In [81]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset)
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

95.86692428588867


In [82]:
train.columns

Index(['service_sales_node_base_sales_hierarchy_level',
       'contract_line_net_usd_amount', 'product_net_price',
       'sales_node_renewal_rate', 'partner_renewal_rate',
       'product_renewal_rate',
       'service_sales_node_installed_base_sales_node_renewal_rate',
       'service_partner_installed_base_partner_renewal_rate',
       'minor_line_yorn_N', 'minor_line_yorn_Y',
       ...
       'service_distributor_base_distributor_name_C321254900CAC4B6824721B6611C15CD8A9A824C66B6C1CA7FC9BDE39ABC5B18',
       'service_distributor_base_distributor_name_C868A0ED0FB7D710C0235B586BDE12FC90AE26BA384782868BB4EE0A6A2FC9B3',
       'service_distributor_base_distributor_name_D496D3E841D7F257C8B8314A580E5728BC034FD09FD4360908B2103860CC728F',
       'service_distributor_base_distributor_name_E15FFA65FCD7A17CF3F1FE025C496E1B5C5D6A5126918C5FFEFFC411240000F5',
       'service_distributor_base_distributor_name_E1B1913F5C4B81CD4DC2C80B2086BE1E90C063FCB34FBF5E3C8E51C2D89A6162',
       'service_dist

In [83]:
#print the new columns we got
train.shape

(6010041, 487)

In [84]:
test.shape

(1888195, 487)

In [24]:
#Replace all NaN left with 0 No nans left
'''t1=time.time()
train_X = train_X.fillna(0)
test_X = test_X.fillna(0)
t2=time.time()
print(t2-t1)'''

0.19629502296447754


In [85]:
#check data quality now
train.isnull().any().any()

False

In [86]:
#check data quality now
test.isnull().any().any()

False

In [87]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

24.365723133087158


In [88]:
#Dimensions of training data
X_train.shape

(4026727, 487)

In [89]:
import gc
del dataset
gc.collect()

1430

In [36]:
#Lets start fitting different models on the data
#1. start with simple Bayes model
t1=time.time()
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gaussian_model = gnb.fit(X_train,y_train)
t2=time.time()
print(t2-t1)

79.03269386291504


In [37]:
# Lets see the prediction on our validation data
t1=time.time()
pred = gaussian_model.predict_proba(X_valid)
t2=time.time()
print(t2-t1)

20.27684736251831


In [38]:
pred[1,]

array([0.99568086, 0.00431914])

In [39]:
from sklearn.metrics import log_loss
log_loss(y_valid,pred[:,1])

9.173194297593184

In [38]:
#find predicted value on test set
pred = gaussian_model.predict_proba(test_X)

In [39]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]

In [40]:
os.chdir('/users/hdpsndbx125')

In [41]:
#very important to sort the values else won't be taken at submission
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')

In [42]:
#persist to a file
submit.to_csv('NaiveBayesGaussian_EntireTraining.csv',index=False)

In [43]:
submit.shape

(1888195, 2)

In [46]:
#fitting a multinomialNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1.0, fit_prior=True)
clf.fit(X_train,y_train)
pred = clf.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

ValueError: Input X must be non-negative

In [ ]:
#find predicted value on test set
pred = clf.predict_proba(test_X)

In [47]:
from sklearn.naive_bayes import BernoulliNB
mnb = MultinomialNB(alpha=1.0)
mnb.fit(X_train,y_train)
pred = mnb.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

ValueError: Input X must be non-negative

In [40]:
from sklearn.linear_model import LogisticRegression as lr
t1=time.time()
lr1=lr(max_iter=100,verbose=1)
lr1.fit(X_train,y_train)
pred = lr1.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])
t2=time.time()
print(t2-t1)

3439.8842487335205


In [90]:
log_loss(y_valid,pred[:,1])

0.5479825664266159

In [42]:
#find predicted value on test set
pred = lr1.predict_proba(test)

In [44]:
test = pd.read_table('/aichallenge/scoring_set.tsv')

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
test.shape

(1888195, 237)

In [46]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['scoring_set.innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('LR_100iter_impute_full.csv',index=False)

In [48]:
submit.shape

(1888195, 2)

In [91]:
del train
gc.collect()

11

In [51]:
from sklearn.linear_model import SGDClassifier
t1=time.time()
sgd = SGDClassifier(loss="log", penalty="l2",max_iter=1000,verbose=True)
sgd.fit(X_train,y_train)
pred = sgd.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

-- Epoch 1
Norm: 4.85, NNZs: 477, Bias: -1.981515, T: 4026727, Avg. loss: 0.965915
Total training time: 8.46 seconds.
-- Epoch 2
Norm: 4.69, NNZs: 477, Bias: -1.979594, T: 8053454, Avg. loss: 0.561289
Total training time: 16.99 seconds.
-- Epoch 3
Norm: 4.67, NNZs: 477, Bias: -1.969140, T: 12080181, Avg. loss: 0.555884
Total training time: 25.52 seconds.
-- Epoch 4
Norm: 4.66, NNZs: 477, Bias: -1.968123, T: 16106908, Avg. loss: 0.553529
Total training time: 34.13 seconds.
-- Epoch 5
Norm: 4.63, NNZs: 477, Bias: -1.962530, T: 20133635, Avg. loss: 0.552382
Total training time: 42.67 seconds.
-- Epoch 6
Norm: 4.62, NNZs: 477, Bias: -1.963317, T: 24160362, Avg. loss: 0.551539
Total training time: 51.18 seconds.
-- Epoch 7
Norm: 4.62, NNZs: 477, Bias: -1.958928, T: 28187089, Avg. loss: 0.551055
Total training time: 59.67 seconds.
-- Epoch 8
Norm: 4.60, NNZs: 477, Bias: -1.960887, T: 32213816, Avg. loss: 0.550703
Total training time: 68.28 seconds.
-- Epoch 9
Norm: 4.61, NNZs: 477, Bias: -1.

Norm: 4.60, NNZs: 477, Bias: -1.941740, T: 273817436, Avg. loss: 0.548473
Total training time: 584.53 seconds.
-- Epoch 69
Norm: 4.60, NNZs: 477, Bias: -1.941389, T: 277844163, Avg. loss: 0.548462
Total training time: 593.05 seconds.
-- Epoch 70
Norm: 4.60, NNZs: 477, Bias: -1.941924, T: 281870890, Avg. loss: 0.548462
Total training time: 601.55 seconds.
-- Epoch 71
Norm: 4.60, NNZs: 477, Bias: -1.941470, T: 285897617, Avg. loss: 0.548451
Total training time: 610.13 seconds.
-- Epoch 72
Norm: 4.60, NNZs: 477, Bias: -1.941017, T: 289924344, Avg. loss: 0.548447
Total training time: 618.72 seconds.
-- Epoch 73
Norm: 4.59, NNZs: 477, Bias: -1.941627, T: 293951071, Avg. loss: 0.548450
Total training time: 627.34 seconds.
-- Epoch 74
Norm: 4.60, NNZs: 477, Bias: -1.941871, T: 297977798, Avg. loss: 0.548444
Total training time: 635.96 seconds.
-- Epoch 75
Norm: 4.59, NNZs: 477, Bias: -1.941410, T: 302004525, Avg. loss: 0.548434
Total training time: 644.52 seconds.
-- Epoch 76
Norm: 4.60, NNZs

Norm: 4.59, NNZs: 477, Bias: -1.936014, T: 543608145, Avg. loss: 0.548327
Total training time: 1159.84 seconds.
-- Epoch 136
Norm: 4.59, NNZs: 477, Bias: -1.936001, T: 547634872, Avg. loss: 0.548326
Total training time: 1168.41 seconds.
-- Epoch 137
Norm: 4.59, NNZs: 477, Bias: -1.936001, T: 551661599, Avg. loss: 0.548329
Total training time: 1176.98 seconds.
-- Epoch 138
Norm: 4.59, NNZs: 477, Bias: -1.936141, T: 555688326, Avg. loss: 0.548325
Total training time: 1185.56 seconds.
-- Epoch 139
Norm: 4.59, NNZs: 477, Bias: -1.935970, T: 559715053, Avg. loss: 0.548316
Total training time: 1194.13 seconds.
-- Epoch 140
Norm: 4.59, NNZs: 477, Bias: -1.936009, T: 563741780, Avg. loss: 0.548323
Total training time: 1202.74 seconds.
-- Epoch 141
Norm: 4.59, NNZs: 477, Bias: -1.936091, T: 567768507, Avg. loss: 0.548323
Total training time: 1211.38 seconds.
-- Epoch 142
Norm: 4.59, NNZs: 477, Bias: -1.935335, T: 571795234, Avg. loss: 0.548322
Total training time: 1220.04 seconds.
-- Epoch 143


Norm: 4.59, NNZs: 477, Bias: -1.932930, T: 809372127, Avg. loss: 0.548280
Total training time: 1724.48 seconds.
-- Epoch 202
Norm: 4.59, NNZs: 477, Bias: -1.933370, T: 813398854, Avg. loss: 0.548282
Total training time: 1733.04 seconds.
-- Epoch 203
Norm: 4.59, NNZs: 477, Bias: -1.933140, T: 817425581, Avg. loss: 0.548278
Total training time: 1741.53 seconds.
-- Epoch 204
Norm: 4.59, NNZs: 477, Bias: -1.933173, T: 821452308, Avg. loss: 0.548284
Total training time: 1750.00 seconds.
-- Epoch 205
Norm: 4.59, NNZs: 477, Bias: -1.932781, T: 825479035, Avg. loss: 0.548279
Total training time: 1758.51 seconds.
-- Epoch 206
Norm: 4.59, NNZs: 477, Bias: -1.933111, T: 829505762, Avg. loss: 0.548276
Total training time: 1766.98 seconds.
-- Epoch 207
Norm: 4.59, NNZs: 477, Bias: -1.933041, T: 833532489, Avg. loss: 0.548281
Total training time: 1775.42 seconds.
-- Epoch 208
Norm: 4.59, NNZs: 477, Bias: -1.932487, T: 837559216, Avg. loss: 0.548282
Total training time: 1783.82 seconds.
-- Epoch 209


Norm: 4.59, NNZs: 477, Bias: -1.930104, T: 1131510287, Avg. loss: 0.548255
Total training time: 2407.88 seconds.
-- Epoch 282
Norm: 4.59, NNZs: 477, Bias: -1.929847, T: 1135537014, Avg. loss: 0.548251
Total training time: 2416.40 seconds.
-- Epoch 283
Norm: 4.59, NNZs: 477, Bias: -1.930266, T: 1139563741, Avg. loss: 0.548255
Total training time: 2424.90 seconds.
-- Epoch 284
Norm: 4.59, NNZs: 477, Bias: -1.930483, T: 1143590468, Avg. loss: 0.548251
Total training time: 2433.38 seconds.
-- Epoch 285
Norm: 4.59, NNZs: 477, Bias: -1.930547, T: 1147617195, Avg. loss: 0.548251
Total training time: 2441.89 seconds.
-- Epoch 286
Norm: 4.59, NNZs: 477, Bias: -1.930192, T: 1151643922, Avg. loss: 0.548256
Total training time: 2450.37 seconds.
-- Epoch 287
Norm: 4.59, NNZs: 477, Bias: -1.930136, T: 1155670649, Avg. loss: 0.548260
Total training time: 2458.88 seconds.
-- Epoch 288
Norm: 4.59, NNZs: 477, Bias: -1.930450, T: 1159697376, Avg. loss: 0.548247
Total training time: 2467.37 seconds.
-- Ep

...

Norm: 4.59, NNZs: 477, Bias: -1.923703, T: 2726094179, Avg. loss: 0.548214
Total training time: 5793.00 seconds.
-- Epoch 678
Norm: 4.59, NNZs: 477, Bias: -1.923536, T: 2730120906, Avg. loss: 0.548211
Total training time: 5801.52 seconds.
-- Epoch 679
Norm: 4.59, NNZs: 477, Bias: -1.923594, T: 2734147633, Avg. loss: 0.548216
Total training time: 5810.08 seconds.
-- Epoch 680
Norm: 4.59, NNZs: 477, Bias: -1.923483, T: 2738174360, Avg. loss: 0.548217
Total training time: 5818.66 seconds.
-- Epoch 681
Norm: 4.59, NNZs: 477, Bias: -1.923520, T: 2742201087, Avg. loss: 0.548215
Total training time: 5827.28 seconds.
-- Epoch 682
Norm: 4.59, NNZs: 477, Bias: -1.923469, T: 2746227814, Avg. loss: 0.548213
Total training time: 5835.87 seconds.
-- Epoch 683
Norm: 4.59, NNZs: 477, Bias: -1.923465, T: 2750254541, Avg. loss: 0.548216
Total training time: 5844.53 seconds.
-- Epoch 684
Norm: 4.59, NNZs: 477, Bias: -1.923292, T: 2754281268, Avg. loss: 0.548216
Total training time: 5853.15 seconds.
-- Ep

Norm: 4.59, NNZs: 477, Bias: -1.922747, T: 2991858161, Avg. loss: 0.548212
Total training time: 6359.93 seconds.
-- Epoch 744
Norm: 4.59, NNZs: 477, Bias: -1.922915, T: 2995884888, Avg. loss: 0.548212
Total training time: 6368.42 seconds.
-- Epoch 745
Norm: 4.59, NNZs: 477, Bias: -1.922824, T: 2999911615, Avg. loss: 0.548210
Total training time: 6377.00 seconds.
-- Epoch 746
Norm: 4.59, NNZs: 477, Bias: -1.922664, T: 3003938342, Avg. loss: 0.548212
Total training time: 6385.53 seconds.
-- Epoch 747
Norm: 4.59, NNZs: 477, Bias: -1.922607, T: 3007965069, Avg. loss: 0.548214
Total training time: 6394.34 seconds.
-- Epoch 748
Norm: 4.59, NNZs: 477, Bias: -1.922791, T: 3011991796, Avg. loss: 0.548209
Total training time: 6402.95 seconds.
-- Epoch 749
Norm: 4.59, NNZs: 477, Bias: -1.922552, T: 3016018523, Avg. loss: 0.548215
Total training time: 6412.07 seconds.
-- Epoch 750
Norm: 4.59, NNZs: 477, Bias: -1.922727, T: 3020045250, Avg. loss: 0.548214
Total training time: 6420.63 seconds.
-- Ep

Norm: 4.59, NNZs: 477, Bias: -1.922137, T: 3257622143, Avg. loss: 0.548211
Total training time: 6927.98 seconds.
-- Epoch 810
Norm: 4.59, NNZs: 477, Bias: -1.922062, T: 3261648870, Avg. loss: 0.548210
Total training time: 6936.49 seconds.
-- Epoch 811
Norm: 4.59, NNZs: 477, Bias: -1.922174, T: 3265675597, Avg. loss: 0.548211
Total training time: 6945.09 seconds.
-- Epoch 812
Norm: 4.59, NNZs: 477, Bias: -1.922273, T: 3269702324, Avg. loss: 0.548212
Total training time: 6953.66 seconds.
-- Epoch 813
Norm: 4.59, NNZs: 477, Bias: -1.922114, T: 3273729051, Avg. loss: 0.548210
Total training time: 6962.40 seconds.
-- Epoch 814
Norm: 4.59, NNZs: 477, Bias: -1.922108, T: 3277755778, Avg. loss: 0.548211
Total training time: 6971.04 seconds.
-- Epoch 815
Norm: 4.59, NNZs: 477, Bias: -1.922170, T: 3281782505, Avg. loss: 0.548212
Total training time: 6979.73 seconds.
-- Epoch 816
Norm: 4.59, NNZs: 477, Bias: -1.922187, T: 3285809232, Avg. loss: 0.548206
Total training time: 6988.37 seconds.
-- Ep

Norm: 4.59, NNZs: 477, Bias: -1.921616, T: 3523386125, Avg. loss: 0.548206
Total training time: 7494.74 seconds.
-- Epoch 876
Norm: 4.59, NNZs: 477, Bias: -1.921465, T: 3527412852, Avg. loss: 0.548210
Total training time: 7503.32 seconds.
-- Epoch 877
Norm: 4.59, NNZs: 477, Bias: -1.921517, T: 3531439579, Avg. loss: 0.548210
Total training time: 7511.97 seconds.
-- Epoch 878
Norm: 4.59, NNZs: 477, Bias: -1.921328, T: 3535466306, Avg. loss: 0.548208
Total training time: 7520.67 seconds.
-- Epoch 879
Norm: 4.59, NNZs: 477, Bias: -1.921542, T: 3539493033, Avg. loss: 0.548210
Total training time: 7529.30 seconds.
-- Epoch 880
Norm: 4.59, NNZs: 477, Bias: -1.921414, T: 3543519760, Avg. loss: 0.548209
Total training time: 7537.83 seconds.
-- Epoch 881
Norm: 4.59, NNZs: 477, Bias: -1.921552, T: 3547546487, Avg. loss: 0.548208
Total training time: 7546.31 seconds.
-- Epoch 882
Norm: 4.59, NNZs: 477, Bias: -1.921314, T: 3551573214, Avg. loss: 0.548208
Total training time: 7554.81 seconds.
-- Ep

Norm: 4.59, NNZs: 477, Bias: -1.921293, T: 3789150107, Avg. loss: 0.548204
Total training time: 8061.79 seconds.
-- Epoch 942
Norm: 4.59, NNZs: 477, Bias: -1.920929, T: 3793176834, Avg. loss: 0.548208
Total training time: 8070.33 seconds.
-- Epoch 943
Norm: 4.59, NNZs: 477, Bias: -1.921249, T: 3797203561, Avg. loss: 0.548209
Total training time: 8078.81 seconds.
-- Epoch 944
Norm: 4.59, NNZs: 477, Bias: -1.920967, T: 3801230288, Avg. loss: 0.548206
Total training time: 8087.26 seconds.
-- Epoch 945
Norm: 4.59, NNZs: 477, Bias: -1.920900, T: 3805257015, Avg. loss: 0.548205
Total training time: 8095.73 seconds.
-- Epoch 946
Norm: 4.59, NNZs: 477, Bias: -1.920938, T: 3809283742, Avg. loss: 0.548209
Total training time: 8104.20 seconds.
-- Epoch 947
Norm: 4.59, NNZs: 477, Bias: -1.920983, T: 3813310469, Avg. loss: 0.548208
Total training time: 8112.89 seconds.
-- Epoch 948
Norm: 4.59, NNZs: 477, Bias: -1.920875, T: 3817337196, Avg. loss: 0.548208
Total training time: 8121.50 seconds.
-- Ep

In [52]:
pred = sgd.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

0.5479825664266159

In [92]:
#find predicted value on test set
pred = sgd.predict_proba(test)
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('SGD_1000iter_impute_full.csv',index=False)

In [93]:
#trying Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
t1=time.time()
rfc = RandomForestClassifier(max_depth=4, random_state=0,verbose=True)
rfc.fit(X_train, y_train)
pred = rfc.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   34.8s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.1s finished


0.5660623015116591
62.94131875038147


In [94]:
pred = rfc.predict_proba(test)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished


In [95]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('RFC_depth4_impute-full.csv',index=False)

In [97]:
clean_col_name=[]
for col in X_train.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_train.columns = clean_col_name
clean_col_name=[]
for col in X_valid.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_valid.columns = clean_col_name

In [102]:
gc.collect()

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f7011857d30>>
Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/xgboost-0.7-py3.6.egg/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f70243239e8>>
Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/xgboost-0.7-py3.6.egg/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


619

In [103]:
#Time for xgboost
import xgboost as xgb
import time
t1 = time.time()
model1 = xgb.XGBClassifier()
#model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)
xgb1 = model1.fit(X_train, y_train)
#xgb2 = model2.fit(X_train.iloc[0:10,], y_train[0:10])
t2 = time.time()

In [104]:
t2-t1

4776.111815214157

In [105]:
pred=xgb1.predict_proba(X_valid)

In [117]:
xgb1

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [77]:
from sklearn.metrics import log_loss
log_loss(y_valid,pred[:,1])

0.5286575244360493

In [109]:
col=test.columns
clean_col_name=[]
for col in test.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
test.columns = clean_col_name

xgb1

In [111]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=xgb1.predict_proba(test)[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-entiretrainscl-impute-full.csv',index=False)

In [115]:
gc.collect()

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f7011857cc0>>
Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/xgboost-0.7-py3.6.egg/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


788

In [113]:
#Trying MLPerceptron
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20),verbose=True)

In [116]:
t1=time.time()
mlp.fit(X_train,y_train)
t2=time.time()
print(t2-t1)

MemoryError: 

In [85]:
from sklearn.metrics import log_loss
pred = mlp.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

0.32762739704396604

In [86]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']

In [87]:
pred=mlp.predict_proba(test_X)

In [88]:
len(pred)

1888195

In [89]:
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('MLP_20_20_20_scl-impute.csv',index=False)